
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>



# LAB - Create a ReAct Agent

In this lab, we will create a virtual stock advisor agent that uses the **ReAct** prompting framework. This agent will reason and act based on the prompts and tools provided.

Of course, the agent is built for demonstration purposes and the answers shouldn't be used as investment advice!


**Lab Outline:**

In this lab, you will need to complete the following tasks;

* **Task 1 :** Define the agent brain

* **Task 2 :** Define the agent tools

* **Task 3 :** Define an agent logic

* **Task 4 :** Create the agent 

* **Task 5 :** Run the agent

**📝 Your task:** Complete the **`<FILL_IN>`** sections in the code blocks and follow the other steps as instructed.

## REQUIRED - SELECT CLASSIC COMPUTE
Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:
1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.
   
   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.

## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **15.4.x-cpu-ml-scala2.12**


## Classroom Setup

Before starting the lab, run the provided classroom setup scripts. These scripts will install the required libraries and configure variables which are needed for the lab.

In [0]:
%pip install -U -qq langchain-databricks langchain==0.3.7 langchain_community==0.3.5 wikipedia==1.4.0 youtube-search duckduckgo-search

dbutils.library.restartPython()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
petastorm 0.12.1 requires pyspark>=2.1.0, which is not installed.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.10.0 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.5.1 requires numpy<1.24,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.5.1 requires pydantic<2,>=1.8.1, but you have pydantic 2.11.7 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%run ../Includes/Classroom-Setup-03LAB

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


No such comm: LSP_COMM_ID


30
30



The examples and models presented in this course are intended solely for demonstration and educational purposes.
 Please note that the models and prompt examples may sometimes contain offensive, inaccurate, biased, or harmful content.


**Enable MLflow Auto-Log**

In [0]:
import mlflow
mlflow.langchain.autolog()

**Other Conventions:**

Throughout this demo, we'll refer to the object `DA`. This object, provided by Databricks Academy, contains variables such as your username, catalog name, schema name, working directory, and dataset locations. Run the code block below to view these details:

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location:  {DA.paths.datasets}")

Username:          labuser11182879_1754974636@vocareum.com
Catalog Name:      dbacademy
Schema Name:       labuser11182879_1754974636
Working Directory: /Volumes/dbacademy/ops/labuser11182879_1754974636@vocareum_com
Dataset Location:  NestedNamespace (dais='/Volumes/dbacademy_dais/v01', docs='/Volumes/dbacademy_docs/v01')


## Task 1: Define the Brain of the Agent

Let's start by defining the agent brain using one of the available LLMs on Databricks.

* For this lab, use **databricks-meta-llama-3-3-70b-instruct** 

* Define the maximum number of tokens

In [0]:
from langchain.agents import AgentType, initialize_agent
from langchain_databricks import ChatDatabricks

## define the brain
llm_llama = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens = 2500)

/root/.ipykernel/8503/command-3554075450802205-465907747:5: LangChainDeprecationWarning: Use databricks_langchain.ChatDatabricks
  llm_llama = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens = 2500)


In [0]:
## let's test the brain
llm_llama.invoke("Hi! How are you?")

AIMessage(content="Hello! I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have! How about you? How's your day going so far?", additional_kwargs={}, response_metadata={'prompt_tokens': 16, 'completion_tokens': 52, 'total_tokens': 68}, id='run--ef86eab3-05be-4561-851b-0e9fa93e9c54-0')

Trace(request_id=tr-02ad5d1c3735416ea3230c2b978b5fcc)

## Task 2 : Define Tools that the Agent Will Use

For an agent, the toolset is a crucial component to complete the defined task. The agent will reason about the next action to take and use an appropriate tool to complete that action.

For our stock advisor agent, we will use the following tools:

* **Wikipedia tool**: For searching company details. [[LangChain Doc](https://python.langchain.com/docs/integrations/tools/wikipedia/)]

* **DuckDuckGo Search tool**: This will be used for searching the latest news and updates about a stock.[[LangChain Doc](https://python.langchain.com/docs/integrations/tools/ddg/)]

* **YouTube search tool**: This will be used to search for review videos about the latest financial reports. [[LangChain Doc](https://python.langchain.com/docs/integrations/tools/youtube/)]

### Wikipedia Tool

In [0]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=1000)
tool_wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [0]:
## Test the Wikipedia tool
tool_wiki.run("Apple company history")

"Page: Apple Inc.\nSummary: Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley. It is best known for its consumer electronics, software, and services. Founded in 1976 as Apple Computer Company by Steve Jobs, Steve Wozniak and Ronald Wayne, the company was incorporated by Jobs and Wozniak as Apple Computer, Inc. the following year. It was renamed Apple Inc. in 2007 as the company had expanded its focus from computers to consumer electronics. Apple is the largest technology company by revenue, with US$391.04 billion in the 2024 fiscal year.\nThe company was founded to produce and market Wozniak's Apple I personal computer. Its second computer, the Apple II, became a best seller as one of the first mass-produced microcomputers. Apple introduced the Lisa in 1983 and the Macintosh in 1984, as some of the first computers to use a graphical user interface and a mouse. By 1985, internal company problems led to Jo

Trace(request_id=tr-92a57c6ee72e4858ad2b4c074f2eb07e)

### DuckDuckGo Search Tool

In [0]:
## DuckDuckGo Search Tool
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

tool_news = DuckDuckGoSearchRun()

In [0]:
## Test DuckDuckGo tool
tool_news.run("Latest news about Apple stock")

/local_disk0/.ephemeral_nfs/envs/pythonEnv-4878a7f2-7fe8-4a6d-a0e4-27bf0d98e559/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/local_disk0/.ephemeral_nfs/envs/pythonEnv-4878a7f2-7fe8-4a6d-a0e4-27bf0d98e559/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


"The latest lottery results, news and information for UK National Lottery draws including Lotto and more. From The National Lottery comes the Lotto Game. Lotto is the original lottery game. It's been loved by millions since 1994. You could become the UK’s latest millionaire by winning the … May 27, 2023 · From The National Lottery comes the Lotto Game. Lotto is the original lottery game. It's been loved by millions since 1994. You could become the UK’s latest millionaire by … The latest Lotto HotPicks results, news and information for UK National Lottery draws including Lotto and more. From The National Lottery comes the Lotto Game. Lotto is the original lottery game. It's been loved by millions since 1994. You could become the UK’s latest millionaire by winning the …"

Trace(request_id=tr-1db3ad1ebde2434dabb53230b3d7b864)

### YouTube Search Tool

In [0]:
from langchain_community.tools import YouTubeSearchTool

tool_youtube = YouTubeSearchTool()

<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()


In [0]:
## Test the YouTube search tool 
tool_youtube.run("Apple company stock")

<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()


"['https://www.youtube.com/watch?v=VZ4dKqocSu0&pp=ygUTQXBwbGUgY29tcGFueSBzdG9jaw%3D%3D', 'https://www.youtube.com/watch?v=yvQluY2wIeU&pp=ygUTQXBwbGUgY29tcGFueSBzdG9jaw%3D%3D']"

Trace(request_id=tr-b17d0f074e064b31811305989e833a8e)

### Define Toolset

Next, add all tools together to be used in the next step.

In [0]:
tools = [tool_wiki, tool_youtube, tool_news]

## Task 3: Define Agent Logic

An agent requires a planning prompt that defines how it should **reason and act**. LangChain provides a few ready-to-use prompt templates through its **[LangChain Hub](https://smith.langchain.com/hub)**, including those that follow the **ReAct (Reason + Act)** style.

In this task, you'll pull a pre-defined ReAct-style prompt from LangChain Hub, which is designed to work with tool-using agents.

> 💡 Using the Hub simplifies your code and ensures you're working with prompt templates that are community-tested and regularly maintained.

📚 [Learn more: LangChain Agent Executor Documentation](https://python.langchain.com/docs/how_to/agent_executor/#create-the-agent)

In [0]:
## Import LangChain Hub
from langchain import hub

## Pull a ReAct-style prompt for function/tool-calling agents
prompt = hub.pull("hwchase17/openai-functions-agent")

## Display the prompt structure
prompt.messages

<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()
/local_disk0/.ephemeral_nfs/envs/pythonEnv-4878a7f2-7fe8-4a6d-a0e4-27bf0d98e559/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Task 4: Create the Agent

The final step is to create the agent using the LLM, planning prompt and toolset.

In [0]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(llm_llama, tools, prompt)
agent_exec  = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

## Task 5: Run the Agent

Let's test the agent!

Note: YouTube search might not return the best results. For better results, you could use a paid API/Service that supports better search.

**Use this prompt:**
<blockquote>
What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.

Format the final answer as HTML.
</blockquote>


In [0]:
response = agent_exec.invoke({"input": 
    """
    What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.

    Format the final answer as HTML.
    """})

displayHTML(response["output"])



> Entering new AgentExecutor chain...


<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()


<p>Investing in Apple stock can be a viable option for those looking to diversify their portfolio with a well-established technology company. As one of the world's largest and most successful companies, Apple has a strong track record of innovation and customer loyalty. However, it's essential to stay up-to-date with the latest news and trends affecting the company's stock performance.</p>

<p>To get an overview of the latest news about Apple, let's check what's currently available on DuckDuckGo:</p>
<function=duckduckgo_search>{"query": "Apple latest news)}</function>

> Finished chain.


Investing in Apple stock can be a viable option for those looking to diversify their portfolio with a well-established technology company. As one of the world's largest and most successful companies, Apple has a strong track record of innovation and customer loyalty. However, it's essential to stay up-to-date with the latest news and trends affecting the company's stock performance. 

 To get an overview of the latest news about Apple, let's check what's currently available on DuckDuckGo: 
 {"query": "Apple latest news)}

Trace(request_id=tr-fcc9bcdd99e9464186f1227fcf116ea8)

It seems like the agent uses company name only when searching for videos. **Let's play with the prompt to get better results.**

In real-world, the agent would be used in a conversational way and we would just chat with the agent to tweak the search query or to adjust the response in a desired format. 

**Use this prompt:**
<blockquote>
What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.

When searching on YouTube use such as "Apple  stock finance investment earnings valuation".

Write a nice summary of the results.

Format the final answer as HTML.
</blockquote>

In [0]:
response = agent_exec.invoke({"input": 
    """
    What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.
    When searching on YouTube use such as "Apple  stock finance investment earnings valuation".

    Write a nice summary of the results.

    Format the final answer as HTML.

    """})
displayHTML(response["output"])



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'Apple latest news'}`




/local_disk0/.ephemeral_nfs/envs/pythonEnv-4878a7f2-7fe8-4a6d-a0e4-27bf0d98e559/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/local_disk0/.ephemeral_nfs/envs/pythonEnv-4878a7f2-7fe8-4a6d-a0e4-27bf0d98e559/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


1 day ago · Apple News and RumorsApple also continues to test the new Siri with its own apps, he said. Apple's own example was an iPhone user asking Siri about their mother's flight and lunch …Apple added Thunderbolt 5 connectivity to its latest M4 Pro and M4 Max machines, …iPhone 16e Can i return an apple product within 14 days if, within the 14 days, it has …AirPods 4 are Apple's newest AirPods with improved sound and comfort. Two models …Otherwise, reliable Black Friday deals every year include those on Apple TV 4K, …Apple Pay is Apple's mobile payment service, which lets users with an iPhone 6 or …Apple says the Pro Display XDR features the industry's best polarizer technology for a …Apple Studio Display Apple's 27-inch 5K display with A13 chip, 12-megapixel …Apple's revamped 4K Apple TV features an A15 Bionic processor, 64GB or 128GB of … 3 days ago · Taking a look back at this week’s news and headlines from across the Apple world, including iPhone 17 launch dates,iPhone 17 Pro pr

Investing in Apple stock can be a viable option for those looking to diversify their portfolio. Recently, Apple has been testing its new Siri with its own apps and has added Thunderbolt 5 connectivity to its latest M4 Pro and M4 Max machines. The company has also released new AirPods with improved sound and comfort, and its Pro Display XDR features the industry's best polarizer technology. 

 In terms of financials, Apple is expected to unveil its iPhone 17 lineup on September 9, with leaks pointing to design changes, upgraded cameras, and possible price increases for the Pro models. The company has also announced that it will be investing $600 billion in the US over the next four years. 

 A video on YouTube reviewing Apple's latest financials can be found at https://www.youtube.com/watch?v=seWLO2qAPcY and https://www.youtube.com/watch?v=HvOrs-mO2Zo. These videos provide an in-depth analysis of Apple's financial performance and offer insights into the company's future prospects. 

 Overall, Apple's latest news and financials suggest that the company is continuing to innovate and invest in its products and services. While there are risks associated with investing in any stock, Apple's strong track record and commitment to innovation make it an attractive option for investors.

Trace(request_id=tr-0bee5ded1adf461b8353d95f0eaba199)


## Conclusion

In this lab, you successfully created a virtual stock advisor agent using the ReAct prompting framework. By defining the agent's brain, tools, logic, and execution flow, you demonstrated how the agent can interact with various data sources to provide investment insights.


&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>